In [2]:
import duckdb
import pandas as pd
import os

In [70]:
db_dir='./database'
os.listdir(db_dir)

['All_Full_reservations.csv', 'All_reservations.csv', 'oldFinal.csv']

## Create database or database connection

In [4]:
# # to start an in-memory database
# con = duckdb.connect(database=':memory:')
# # to use a database file (not shared between processes)
con = duckdb.connect(database=db_dir+'/All_reservations.duckdb', read_only=False) # if doesn't exist
# # to use a database file (shared between processes)
# con = duckdb.connect(database='All_Full_reservations.duckdb', read_only=True) # if exists

### Import data as dataframe

In [5]:
res_df = pd.read_csv(db_dir+'/All_reservations.csv')

### create the table "curr_reservations" from the DataFrame "res_df"

In [6]:
con.execute("CREATE TABLE curr_reservations AS SELECT * FROM res_df")

### Create a relation to the table to be used with other functions

In [7]:
all_res = con.table('curr_reservations')

In [ ]:
curr

In [ ]:
print(curr.columns)

### filter 

In [ ]:
print(curr.filter('Nights > 10'))

### Save query results as a dataframe

In [ ]:
long_stays=curr.filter('Nights > 10').to_df()

In [ ]:
long_stays

In [ ]:
# # create a table in duckdb from the relation
# print(rel.create("test_table2"))

# # insert the relation's data into an existing table
# conn.execute("CREATE TABLE test_table3 (i INTEGER, j STRING)")
# print(rel.insert_into("test_table3"))

# Create GrandWelcome Scenarios

### 1 Scrape web table and save as curr_online table 

In [5]:
# Broken update. Seems to be running but not updating. 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import Select
import pandas as pd
from time import sleep
from datetime import datetime
from datetime import timedelta
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
import json
# import get_params as gp
# import chk_new_res as cn
# import build_payload as bp
# import send2cal as cal
import logging
import re
from dateutil.parser import parse
from datetime import date, timedelta
# logging.basicConfig(filename='logs/booking.log', level=logging.INFO)

def check_new(df):
    if len(df.index) == 0:
        logging.info('No new booking found.')
    else:
        logging.info('New booking found.')

def CountDaysMonth(Arrival,Departure):    
    from datetime import date, timedelta
    from dateutil.parser import parse
    from datetime import datetime
    BookedDaysPerMonth = {}
    d1 = parse(Arrival).date()
    d2 = parse(Departure).date()
    # this will give you a list containing all of the dates
    times = [d1 + timedelta(days=x) for x in range((d2-d1).days + 1)]
    # Creates a dictionary and increments if it exists 
    for t in times:
        month = t.strftime("%b")+'-'+t.strftime("%Y")
        if month in BookedDaysPerMonth:
            BookedDaysPerMonth[month] += 1
        else:
            BookedDaysPerMonth[month] = 1
    Dates_list=[]
    for k,v in BookedDaysPerMonth.items():
        Dates_list.append('{}-{}'.format(k,v))
    return Dates_list

In [6]:
url='https://gw.trackhs.com/owner/'
user='4806BrierRose@brenesfinancial.com'
pwd='MyM0n3yH0m3'   

    
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#     browser = webdriver.Remote('http://bookings:4444/wd/hub', desired_capabilities=DesiredCapabilities.CHROME)
browser.get(url)

# Login 
username=browser.find_element(by=By.XPATH, value="//input[@placeholder='Email']")
password=browser.find_element(by=By.XPATH, value="//input[@placeholder='Password']")
login=browser.find_element(by=By.XPATH, value="//button[@type='submit']")
username.send_keys(user)
password.send_keys(pwd)
login.submit()

sleep(5)
## click reservations tab
restab=browser.find_element(by=By.XPATH, value="//span[normalize-space()='Reservations']")
restab.click()
sleep(5)

## Need to make sure that 100 is showing instead of 10
select = Select(browser.find_element(by=By.XPATH, value="//*[@id='table_length']/label/select"))
# select by value 
select.select_by_value('100')
sleep(5) #if you don't wait it will only grab 10
# Get table 
html = browser.page_source
table = pd.read_html(html)
res_raw_df = table[0]
res_df=res_raw_df[['Res. #', 'Status', 'Unit', 'Guest', 'Booked Date', 'Check-In','Checkout', 'Nights', 'Income']]
res_df.columns = ['Res_ID', 'Status', 'Unit', 'Guest', 'Booked Date', 'Check-In','Checkout', 'Nights', 'Income']

In [7]:
# format dates to match for merging
res_df=res_df.copy()
res_df["Booked Date"] = pd.to_datetime(res_df["Booked Date"]).dt.strftime('%b/%d/%Y')
res_df["Check-In"] = pd.to_datetime(res_df["Check-In"]).dt.strftime('%b/%d/%Y')
res_df["Checkout"] = pd.to_datetime(res_df["Checkout"]).dt.strftime('%b/%d/%Y')
# new_res=res_df[['Res_ID','Guest','Check-In','Checkout','Nights','Income']]

In [23]:
con.execute("CREATE TABLE curr_online AS SELECT * FROM res_df")
online = con.table('curr_online')

# Table Names and structure
    **online** This is the table that is scraped at run time.  
        And dropped at end of run.
    **gw_res** This is the table that has all the reservations from gr.  
        This gets updated. 
    **cfrm** This is the old reservations from cfrm in gw_res format.  
        Gets added at initial creation of 'gw_res'

In [68]:
gw_con.close()

In [24]:
gw_con.execute("show tables").fetchall()

[('breakdown_dashboard',), ('cfrm',), ('gw_res',)]

In [25]:
# create gwdb
# gw_con = duckdb.connect(database='database/grandwelcome.duckdb', read_only=False) 
# # create gw_res table
# gw_con.execute("CREATE TABLE gw_res AS SELECT * FROM res_df")
# # create cfrm from csv 
# cfrm_df = pd.read_csv('database/oldFinal.csv')
# gw_con.execute("CREATE TABLE cfrm AS SELECT * FROM cfrm_df")

## Check NEw

In [12]:
def checkNewRes(res_df):
    # create gwdb
    gw_con = duckdb.connect(database='database/grandwelcome.duckdb', read_only=False) 
    gw_con.execute("CREATE TABLE online AS SELECT * FROM res_df")
    new=gw_con.execute("Select * from online where Res_ID NOT IN (Select Res_ID from gw_res)").df()   
    gw_con.execute("drop table IF EXISTS online")
    if len(new) > 0:
        addnew = gw_con.df(new)
        # add new to database 
        addnew.insert_into("gw_res")        
    return new

In [13]:
gw_con.execute("Show tables").fetchall()

[('cfrm',), ('gw_res',)]

In [27]:
result=checkNewRes(res_df)
result

,Res_ID,Status,Unit,Guest,Booked Date,Check-In,Checkout,Nights,Income


In [18]:
result

,Res_ID,Status,Unit,Guest,Booked Date,Check-In,Checkout,Nights,Income


In [27]:
gw_con.close()

In [28]:
gw_con.execute("Select * from gw_res").df()

,Res_ID,Status,Unit,Guest,Booked Date,Check-In,Checkout,Nights,Income
0,1096275,Checked Out,Legendary Times at Story Lake,Brenes,Aug/24/2022,Sep/23/2022,Oct/02/2022,9,$0.00
1,1096930,Checked Out,Legendary Times at Story Lake,Sarousi,Aug/31/2022,Oct/14/2022,Oct/19/2022,5,$562.27
2,1097362,Checked Out,Legendary Times at Story Lake,Cutcher,Sep/05/2022,Oct/06/2022,Oct/12/2022,6,$674.73
3,1098288,Confirmed,Legendary Times at Story Lake,Hool,Sep/16/2022,Feb/01/2023,Feb/05/2023,4,$468.18
4,1098795,Checked Out,Legendary Times at Story Lake,Reed,Sep/22/2022,Oct/22/2022,Oct/30/2022,8,$899.64
5,1099938,Confirmed,Legendary Times at Story Lake,rippy,Oct/05/2022,Dec/23/2022,Jan/02/2023,10,"$2,582.61"
6,1100775,Confirmed,Legendary Times at Story Lake,Russell,Oct/14/2022,Nov/23/2022,Nov/26/2022,3,$506.43
7,1102386,Checked In,Legendary Times at Story Lake,Philbrook,Nov/01/2022,Nov/19/2022,Nov/23/2022,4,$421.51
8,1102644,Checked Out,Legendary Times at Story Lake,Figueroa,Nov/03/2022,Nov/06/2022,Nov/09/2022,3,$314.41
9,1102691,Confirmed,Legendary Times at Story Lake,Moore,Nov/04/2022,Dec/03/2022,Dec/10/2022,7,$733.63


In [24]:
con.execute("Select * from curr_online  where Res_ID NOT IN (Select Res_ID from curr_reservations)").df()

,Res_ID,Status,Unit,Guest,Booked Date,Check-In,Checkout,Nights,Income


In [20]:
# drop the temp table so it can be run again later. 
con.execute("drop table IF EXISTS curr_online")

In [28]:
online=con.execute("Select * from curr_online").df()

In [26]:
cfrm_df = pd.read_csv(db_dir+'/oldFinal.csv')
con.execute("CREATE TABLE cfrm AS SELECT * FROM cfrm_df")

In [32]:
old=gw_con.execute("Select * from cfrm").df()

In [31]:
addcf = gw_con.df(cfrm_df)
addcf.insert_into("cfrm")

In [34]:
# old

In [17]:
addold = gw_con.df(old)
# add new to database 
addold.insert_into("gw_res")

In [35]:
gw_con.execute("Select * from gw_res").df()

,Res_ID,Status,Unit,Guest,Booked Date,Check-In,Checkout,Nights,Income
0,1096275,Checked Out,Legendary Times at Story Lake,Brenes,Aug/24/2022,Sep/23/2022,Oct/02/2022,9,$0.00
1,1096930,Checked Out,Legendary Times at Story Lake,Sarousi,Aug/31/2022,Oct/14/2022,Oct/19/2022,5,$562.27
2,1097362,Checked Out,Legendary Times at Story Lake,Cutcher,Sep/05/2022,Oct/06/2022,Oct/12/2022,6,$674.73
3,1098288,Confirmed,Legendary Times at Story Lake,Hool,Sep/16/2022,Feb/01/2023,Feb/05/2023,4,$468.18
4,1098795,Checked Out,Legendary Times at Story Lake,Reed,Sep/22/2022,Oct/22/2022,Oct/30/2022,8,$899.64
5,1099938,Confirmed,Legendary Times at Story Lake,rippy,Oct/05/2022,Dec/23/2022,Jan/02/2023,10,"$2,582.61"
6,1100775,Confirmed,Legendary Times at Story Lake,Russell,Oct/14/2022,Nov/23/2022,Nov/26/2022,3,$506.43
7,1102386,Checked In,Legendary Times at Story Lake,Philbrook,Nov/01/2022,Nov/19/2022,Nov/23/2022,4,$421.51
8,1102644,Checked Out,Legendary Times at Story Lake,Figueroa,Nov/03/2022,Nov/06/2022,Nov/09/2022,3,$314.41
9,1102691,Confirmed,Legendary Times at Story Lake,Moore,Nov/04/2022,Dec/03/2022,Dec/10/2022,7,$733.63


# SAVE!!!

In [36]:
gw_con.execute("Export DATABASE 'database'")

In [21]:
gw_con.close()